<a href="https://colab.research.google.com/github/Malik-Raheel/data_analysis/blob/main/OBV_ATR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

OBV ATR strategy for false breakout, exit strategy, and entries.
daily 10-20 signals for trading using mianly ATR OBV for liquidity, momentum, and daily price fluctuations

In [1]:
pip install ccxt ta requests pandas numpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.6/288.6 kB 20.3 MB/s eta 0:00:00
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=17980242ca2fabdbde39570984271d3bd2749c3a62befc533ce6b6189f8258b3
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.11
    Uninstalling aiohttp-3.11.11:
      Successfully uninstalled aiohttp-3.11.11


In [2]:
import pandas as pd
import numpy as np
import ccxt
import requests
from ta.volatility import AverageTrueRange
from ta.volume import OnBalanceVolumeIndicator

# Discord Webhook for Alerts
DISCORD_WEBHOOK_URL = "https://discord.com/api/webhooks/1331623264031867011/3dTh4xQtlWRV8FXA3bKPiJQDXLhirOx0aoPUG0whSF41S3bXGKjltmUMfhRBm8gnowDz"  # Replace with your webhook URL

# Initialize Exchange
exchange = ccxt.kucoin()  # Replace with your exchange
assets = ["BTC/USDT", "AVAX/USDT", "SOL/USDT", "SLIM/USDT", "XAVA/USDT",
          "LINK/USDT", "CRV/USDT", "CFX/USDT", "ADA/USDT", "XRP/USDT", "INJ/USDT"]

# Fetch historical data
def fetch_data(symbol, timeframe="1h", limit=500):
    """Fetch OHLCV data for a symbol."""
    try:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
        df = pd.DataFrame(ohlcv, columns=["time", "open", "high", "low", "close", "volume"])
        df["time"] = pd.to_datetime(df["time"], unit="ms")
        return df
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

# Calculate OBV and ATR
def calculate_indicators(df):
    """Calculate OBV and ATR indicators."""
    try:
        # Calculate OBV
        obv_indicator = OnBalanceVolumeIndicator(close=df["close"], volume=df["volume"])
        df["OBV"] = obv_indicator.on_balance_volume()

        # Calculate ATR
        atr_indicator = AverageTrueRange(high=df["high"], low=df["low"], close=df["close"], window=14)
        df["ATR"] = atr_indicator.average_true_range()

        return df
    except Exception as e:
        print(f"Error calculating indicators: {e}")
        return df

# Generate signals
def generate_signals(symbol, df):
    """Generate signals based on OBV and ATR."""
    signals = []

    # Ensure sufficient data
    if len(df) < 20:
        return signals

    # Check OBV and ATR trends
    obv_diff = df["OBV"].iloc[-1] - df["OBV"].iloc[-20]
    atr_diff = df["ATR"].iloc[-1] - df["ATR"].iloc[-20]

    # Signal conditions
    if obv_diff > 0 and atr_diff > 0:
        signals.append({
            "symbol": symbol,
            "action": "Breakout Confirmation",
            "message": "OBV and ATR both increasing, strong breakout likely."
        })
    elif obv_diff > 0 and atr_diff <= 0:
        signals.append({
            "symbol": symbol,
            "action": "Caution",
            "message": "OBV increasing but ATR low, breakout may lack momentum."
        })
    elif obv_diff <= 0 and atr_diff > 0:
        signals.append({
            "symbol": symbol,
            "action": "Reversal Signal",
            "message": "ATR increasing but OBV declining, potential trend reversal."
        })

    return signals

# Send alerts to Discord
def send_discord_alert(signal):
    """Send a signal alert to Discord."""
    message = f"🔔 Signal for {signal['symbol']}:\nAction: {signal['action']}\nDetails: {signal['message']}"
    payload = {"content": message}
    headers = {"Content-Type": "application/json"}
    response = requests.post(DISCORD_WEBHOOK_URL, json=payload, headers=headers)
    if response.status_code == 204:
        print(f"Signal sent: {signal['symbol']} - {signal['action']}")
    else:
        print(f"Failed to send signal: {response.status_code}, {response.text}")

# Main Execution
def main():
    all_signals = []
    for asset in assets:
        data = fetch_data(asset, timeframe="1h")
        if data is not None:
            data = calculate_indicators(data)
            signals = generate_signals(asset, data)
            all_signals.extend(signals)

            # Send signals to Discord
            for signal in signals:
                send_discord_alert(signal)

    # Print all signals
    print(f"Total Signals Generated: {len(all_signals)}")
    for signal in all_signals:
        print(f"{signal['symbol']} - {signal['action']}: {signal['message']}")

# Run strategy every day (example using Colab scheduler or loop)
if __name__ == "__main__":
    main()


Signal sent: SOL/USDT - Caution
Signal sent: XAVA/USDT - Caution
Signal sent: CRV/USDT - Caution
Signal sent: CFX/USDT - Caution
Signal sent: INJ/USDT - Caution
Total Signals Generated: 5
SOL/USDT - Caution: OBV increasing but ATR low, breakout may lack momentum.
XAVA/USDT - Caution: OBV increasing but ATR low, breakout may lack momentum.
CRV/USDT - Caution: OBV increasing but ATR low, breakout may lack momentum.
CFX/USDT - Caution: OBV increasing but ATR low, breakout may lack momentum.
INJ/USDT - Caution: OBV increasing but ATR low, breakout may lack momentum.
